In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import dfm

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_DFM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
        x = self.embed(x)
        # x = x.view(-1,self.embed_output_dim)
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square    
        ix = torch.sum(ix, dim=1, keepdim=True)  
        return self.mlp(x.view(-1,self.embed_output_dim)), 0.5 * ix
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      redency_part = x[redency_part_slice] 
      unredency_part = x[unredency_part_slice] 
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      redency_embed_square_sum = torch.sum(redency_embed ** 2,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_embed ** 2,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return self.redency_linear(redency_embed.view(-1,self.redency_weight_len)) + self.unredency_linear(unredency_embed.view(-1,self.unredency_weight_len)), 0.5 * ix
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_dfm(num_field, prefix,dim = 64,l = [1024,512,256]):
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  DFM_model_ori = dfm.DeepFactorizationMachineModel([100 for i in range(num_field)],dim,l,0.1)
  ori_traced = symbolic_trace(DFM_model_ori)
  
  DFM_model_modify = dfm.DeepFactorizationMachineModel([100 for i in range(num_field)],dim,l,0.1)
  modify_traced = symbolic_trace(DFM_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_DFM(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [14]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [15]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_dfm,l = [1024,512,256],enale = True):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
    return prof
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l)
  if enale:
    p1 = run(ori)
  p2 = run(modify)
  return p1, p2

In [6]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/dfm/dfm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/dfm/dfm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_dfm(num_field = num_field,prefix = prefix,  dim = dim)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [8]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5),(34,29),(22 ,10 )]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


...

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


In [72]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 755 / 1024 (73.7%)
Greatest absolute difference: 0.00474393367767334 at index (90,) (up to 1e-05 allowed)
Greatest relative difference: 0.019195111155238122 at index (81,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:29:22 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:29:23 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:29:23 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.4490275382995605, 0.27741162495544813)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 738 / 1024 (72.1%)
Greatest absolute difference: 0.0033551156520843506 at index (926,) (up to 1e-05 allowed)
Greatest relative difference: 0.01546031178048467 at index (655,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:29:23 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:29:24 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:29:24 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.097372055053711, 0.32583208039795863)


(<torch.profiler.profiler.profile at 0x7fad7fd37400>,
 <torch.profiler.profiler.profile at 0x7fad7f2970d0>)

In [73]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1503 / 2048 (73.4%)
Greatest absolute difference: 0.004145920276641846 at index (318,) (up to 1e-05 allowed)
Greatest relative difference: 0.019176971901149507 at index (1385,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:29:46 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:29:47 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:29:47 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.505481719970703, 0.6619352376674215)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1486 / 2048 (72.6%)
Greatest absolute difference: 0.004938602447509766 at index (1923,) (up to 1e-05 allowed)
Greatest relative difference: 0.021045468481895917 at index (1512,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:29:48 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:29:48 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:29:48 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.067859649658203, 0.5237146146100713)


(<torch.profiler.profiler.profile at 0x7fadb86d1e20>,
 <torch.profiler.profiler.profile at 0x7fadb59a54c0>)

In [74]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2996 / 4096 (73.1%)
Greatest absolute difference: 0.0049596428871154785 at index (1518,) (up to 1e-05 allowed)
Greatest relative difference: 0.017663423088451622 at index (175,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:30:00 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:30:00 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:30:00 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(6.995503902435303, 0.7726403962976747)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3053 / 4096 (74.5%)
Greatest absolute difference: 0.004554629325866699 at index (810,) (up to 1e-05 allowed)
Greatest relative difference: 0.019164036395008993 at index (2934,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:30:01 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:30:02 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:30:02 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(6.463272571563721, 0.9789613355394522)


(<torch.profiler.profiler.profile at 0x7fadb469d4f0>,
 <torch.profiler.profiler.profile at 0x7fadaf602910>)

In [77]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5771 / 8192 (70.4%)
Greatest absolute difference: 0.0041730403900146484 at index (281,) (up to 1e-05 allowed)
Greatest relative difference: 0.01801485641102358 at index (1103,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:31:48 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:31:51 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:31:51 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(35.11406183242798, 1.5555262602617859)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5956 / 8192 (72.7%)
Greatest absolute difference: 0.004303991794586182 at index (6203,) (up to 1e-05 allowed)
Greatest relative difference: 0.016892436432399613 at index (5511,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:31:52 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:31:56 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:31:56 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(39.811320304870605, 1.375575545330321)


(<torch.profiler.profiler.profile at 0x7fad9b3350a0>,
 <torch.profiler.profiler.profile at 0x7fad9b69edc0>)

In [36]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 753 / 1024 (73.5%)
Greatest absolute difference: 0.004382312297821045 at index (552,) (up to 1e-05 allowed)
Greatest relative difference: 0.019369944163362775 at index (262,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:25 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:26 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:26 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.099750518798828, 0.8832018795601471)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 732 / 1024 (71.5%)
Greatest absolute difference: 0.003508627414703369 at index (769,) (up to 1e-05 allowed)
Greatest relative difference: 0.022439741924437453 at index (1000,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:27 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:27 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:27 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.578925132751465, 0.9054738490021919)


(<torch.profiler.profiler.profile at 0x7fb5c74a06d0>,
 <torch.profiler.profiler.profile at 0x7fb5c5fa4520>)

In [40]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1546 / 2048 (75.5%)
Greatest absolute difference: 0.006011009216308594 at index (1392,) (up to 1e-05 allowed)
Greatest relative difference: 0.02122214944653213 at index (795,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:57 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:58 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:58 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.388081073760986, 0.47600441504869195)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1475 / 2048 (72.0%)
Greatest absolute difference: 0.004566371440887451 at index (1488,) (up to 1e-05 allowed)
Greatest relative difference: 0.02015914149780007 at index (1613,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:58 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:59 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:59 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.518256187438965, 0.39492836074259685)


(<torch.profiler.profiler.profile at 0x7fb4a3123fd0>,
 <torch.profiler.profiler.profile at 0x7fb4a13516d0>)

In [39]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2925 / 4096 (71.4%)
Greatest absolute difference: 0.005259722471237183 at index (2512,) (up to 1e-05 allowed)
Greatest relative difference: 0.026282752065868876 at index (4063,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:51 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:52 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:52 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.123902797698975, 1.1040980524001043)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2934 / 4096 (71.6%)
Greatest absolute difference: 0.004484891891479492 at index (1795,) (up to 1e-05 allowed)
Greatest relative difference: 0.021514514184002995 at index (3054,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:13:53 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:13:54 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:13:54 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.658640384674072, 2.7839972340814256)


(<torch.profiler.profiler.profile at 0x7fb5be7df400>,
 <torch.profiler.profiler.profile at 0x7fb4a4549520>)

In [42]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :8192
now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5908 / 8192 (72.1%)
Greatest absolute difference: 0.004802286624908447 at index (7919,) (up to 1e-05 allowed)
Greatest relative difference: 0.021184537601070023 at index (4126,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:14:28 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:14:33 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:14:33 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(54.64937210083008, 16.65027433055002)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5958 / 8192 (72.7%)
Greatest absolute difference: 0.004466652870178223 at index (4911,) (up to 1e-05 allowed)
Greatest relative difference: 0.01870125971348071 at index (684,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:14:34 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:14:39 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:14:39 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(43.61194133758545, 12.500648502532385)


(<torch.profiler.profiler.profile at 0x7fb36cda7880>,
 <torch.profiler.profiler.profile at 0x7fb36c082f10>)

In [44]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 649 / 1024 (63.4%)
Greatest absolute difference: 0.0038940906524658203 at index (71,) (up to 1e-05 allowed)
Greatest relative difference: 0.01968658943111245 at index (909,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:14:50 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:14:50 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:14:50 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.833954334259033, 0.2883378611556964)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 650 / 1024 (63.5%)
Greatest absolute difference: 0.003545403480529785 at index (930,) (up to 1e-05 allowed)
Greatest relative difference: 0.017003238592656112 at index (126,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:14:51 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:14:51 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:14:51 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5202503204345703, 0.3956946386551863)


(<torch.profiler.profiler.profile at 0x7fb24f64d3d0>,
 <torch.profiler.profiler.profile at 0x7fb24edda9d0>)

In [46]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1322 / 2048 (64.6%)
Greatest absolute difference: 0.0034781694412231445 at index (2000,) (up to 1e-05 allowed)
Greatest relative difference: 0.0204635291045942 at index (699,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:15:04 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:15:04 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:15:04 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.94389533996582, 0.2834014503605431)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1271 / 2048 (62.1%)
Greatest absolute difference: 0.003667175769805908 at index (593,) (up to 1e-05 allowed)
Greatest relative difference: 0.015149313505472854 at index (2041,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:15:05 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:15:05 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:15:05 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.0438103675842285, 0.38579208473379367)


(<torch.profiler.profiler.profile at 0x7fb24a4f18b0>,
 <torch.profiler.profiler.profile at 0x7fb248a5f910>)

In [47]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2523 / 4096 (61.6%)
Greatest absolute difference: 0.004875749349594116 at index (630,) (up to 1e-05 allowed)
Greatest relative difference: 0.022785613154652218 at index (4049,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:15:10 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:15:10 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:15:10 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(9.107744693756104, 0.37779631995249474)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2600 / 4096 (63.5%)
Greatest absolute difference: 0.004161477088928223 at index (3512,) (up to 1e-05 allowed)
Greatest relative difference: 0.01820795778653275 at index (3120,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:15:15 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:15:15 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:15:15 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(7.131965160369873, 0.625839149569174)


(<torch.profiler.profiler.profile at 0x7fb247501820>,
 <torch.profiler.profiler.profile at 0x7fb2458e7a30>)

In [49]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5059 / 8192 (61.8%)
Greatest absolute difference: 0.004025876522064209 at index (2578,) (up to 1e-05 allowed)
Greatest relative difference: 0.02345593992998083 at index (2872,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:06 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:11 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:11 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(46.25039339065552, 3.2269919691941595)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 5259 / 8192 (64.2%)
Greatest absolute difference: 0.0055689215660095215 at index (6162,) (up to 1e-05 allowed)
Greatest relative difference: 0.021662142566074905 at index (3814,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:11 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:15 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:15 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(39.51195240020752, 0.638263552468743)


(<torch.profiler.profiler.profile at 0x7fb241590a30>,
 <torch.profiler.profiler.profile at 0x7fb23f81d490>)

In [50]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 642 / 1024 (62.7%)
Greatest absolute difference: 0.004288196563720703 at index (911,) (up to 1e-05 allowed)
Greatest relative difference: 0.016320477297939173 at index (594,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:30 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:30 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:30 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.408124923706055, 0.23453489902749425)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 621 / 1024 (60.6%)
Greatest absolute difference: 0.0038976669311523438 at index (405,) (up to 1e-05 allowed)
Greatest relative difference: 0.01678496181090735 at index (153,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:31 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:31 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:31 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5536599159240723, 0.3569816916012769)


(<torch.profiler.profiler.profile at 0x7fb23d61be20>,
 <torch.profiler.profiler.profile at 0x7fb23c6bccd0>)

In [51]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1311 / 2048 (64.0%)
Greatest absolute difference: 0.0038793087005615234 at index (991,) (up to 1e-05 allowed)
Greatest relative difference: 0.014965588702258089 at index (981,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:38 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:39 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:39 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(7.8679704666137695, 0.5536936671660442)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1275 / 2048 (62.3%)
Greatest absolute difference: 0.0037856698036193848 at index (146,) (up to 1e-05 allowed)
Greatest relative difference: 0.01715446384080896 at index (856,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:40 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:40 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:40 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.777867794036865, 0.389527256237443)


(<torch.profiler.profiler.profile at 0x7fb23b1c1b20>,
 <torch.profiler.profiler.profile at 0x7fb2395b6d90>)

In [52]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2630 / 4096 (64.2%)
Greatest absolute difference: 0.003836333751678467 at index (3265,) (up to 1e-05 allowed)
Greatest relative difference: 0.017477055273931515 at index (2577,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:48 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:51 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:51 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(24.823431968688965, 2.5761275332797595)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2670 / 4096 (65.2%)
Greatest absolute difference: 0.0059999823570251465 at index (2363,) (up to 1e-05 allowed)
Greatest relative difference: 0.022820764535219102 at index (1721,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:19:52 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:19:53 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:19:53 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(9.392127990722656, 1.7284837023453292)


(<torch.profiler.profiler.profile at 0x7fb236be6460>,
 <torch.profiler.profiler.profile at 0x7fb23641cee0>)

In [53]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :8192
now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4911 / 8192 (59.9%)
Greatest absolute difference: 0.004495024681091309 at index (7119,) (up to 1e-05 allowed)
Greatest relative difference: 0.0223418272969707 at index (2260,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:20:00 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:20:07 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:20:07 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(64.39488172531128, 3.470280283062266)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4884 / 8192 (59.6%)
Greatest absolute difference: 0.0031461119651794434 at index (3488,) (up to 1e-05 allowed)
Greatest relative difference: 0.01378385183058399 at index (1127,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:20:07 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:20:12 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:20:12 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(41.08032941818237, 1.313804991576717)


(<torch.profiler.profiler.profile at 0x7fb23492a310>,
 <torch.profiler.profiler.profile at 0x7fb2332bf8e0>)

In [54]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 415 / 1024 (40.5%)
Greatest absolute difference: 0.0026674866676330566 at index (246,) (up to 1e-05 allowed)
Greatest relative difference: 0.01362743574409423 at index (64,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:20:55 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:20:56 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:20:56 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(7.158076763153076, 0.28480275133802024)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 406 / 1024 (39.6%)
Greatest absolute difference: 0.0032486915588378906 at index (365,) (up to 1e-05 allowed)
Greatest relative difference: 0.011990737241374571 at index (221,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:01 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:02 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:02 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.969111919403076, 0.5932426709648553)


(<torch.profiler.profiler.profile at 0x7fb231f24730>,
 <torch.profiler.profiler.profile at 0x7fb230034eb0>)

In [55]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 782 / 2048 (38.2%)
Greatest absolute difference: 0.003211081027984619 at index (1844,) (up to 1e-05 allowed)
Greatest relative difference: 0.014360563714721171 at index (899,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:07 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:08 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:08 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.631536483764648, 0.11391261523385765)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 760 / 2048 (37.1%)
Greatest absolute difference: 0.005484044551849365 at index (716,) (up to 1e-05 allowed)
Greatest relative difference: 0.018406370573720087 at index (53,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:08 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:09 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:09 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.71203899383545, 0.6937711234741073)


(<torch.profiler.profiler.profile at 0x7fb22e700610>,
 <torch.profiler.profiler.profile at 0x7fb0d1f7a4f0>)

In [56]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1543 / 4096 (37.7%)
Greatest absolute difference: 0.003552079200744629 at index (77,) (up to 1e-05 allowed)
Greatest relative difference: 0.012993162338287696 at index (925,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:15 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:18 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:18 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(33.87290000915527, 2.9476798080395383)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1547 / 4096 (37.8%)
Greatest absolute difference: 0.0036314725875854492 at index (1014,) (up to 1e-05 allowed)
Greatest relative difference: 0.018036535419383617 at index (2270,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:19 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:21 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:21 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.097177982330322, 1.1789663146544171)


(<torch.profiler.profiler.profile at 0x7fb0cf4088e0>,
 <torch.profiler.profiler.profile at 0x7fb0cee58ee0>)

In [57]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3133 / 8192 (38.2%)
Greatest absolute difference: 0.005325675010681152 at index (7099,) (up to 1e-05 allowed)
Greatest relative difference: 0.018082774799450873 at index (4802,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:27 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:36 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:36 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(82.48090267181396, 11.811078049936441)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3127 / 8192 (38.2%)
Greatest absolute difference: 0.004117429256439209 at index (4644,) (up to 1e-05 allowed)
Greatest relative difference: 0.01956910324337953 at index (8170,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:37 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:43 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:43 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(66.5936541557312, 2.8801362469209835)


(<torch.profiler.profiler.profile at 0x7faf51321f40>,
 <torch.profiler.profiler.profile at 0x7faf4f653c40>)

In [58]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 404 / 1024 (39.5%)
Greatest absolute difference: 0.0035170912742614746 at index (738,) (up to 1e-05 allowed)
Greatest relative difference: 0.014181571344510492 at index (979,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:49 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:50 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:50 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.71088457107544, 1.0497524277695902)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 378 / 1024 (36.9%)
Greatest absolute difference: 0.0033431053161621094 at index (515,) (up to 1e-05 allowed)
Greatest relative difference: 0.013334060370938887 at index (439,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:51 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:21:52 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:21:52 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.554089069366455, 1.387337120348775)


(<torch.profiler.profiler.profile at 0x7faf4e1e8fa0>,
 <torch.profiler.profiler.profile at 0x7faf4c56d4c0>)

In [59]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 816 / 2048 (39.8%)
Greatest absolute difference: 0.0044689178466796875 at index (1168,) (up to 1e-05 allowed)
Greatest relative difference: 0.022602972446680865 at index (1960,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:21:57 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:00 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:00 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(32.16128349304199, 2.0754288607349736)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 790 / 2048 (38.6%)
Greatest absolute difference: 0.0046163201332092285 at index (1188,) (up to 1e-05 allowed)
Greatest relative difference: 0.015683795756308242 at index (862,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:01 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:03 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:03 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.39018964767456, 0.20005788207413389)


(<torch.profiler.profiler.profile at 0x7faf4b070760>,
 <torch.profiler.profiler.profile at 0x7faf493a2490>)

In [60]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1576 / 4096 (38.5%)
Greatest absolute difference: 0.003909051418304443 at index (3366,) (up to 1e-05 allowed)
Greatest relative difference: 0.019542309810773942 at index (2752,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:04 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:10 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:10 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(59.848177433013916, 3.1605379312793502)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1560 / 4096 (38.1%)
Greatest absolute difference: 0.004072368144989014 at index (1905,) (up to 1e-05 allowed)
Greatest relative difference: 0.018765404838683008 at index (3310,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:11 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:14 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:14 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(36.84602975845337, 2.002745987255139)


(<torch.profiler.profiler.profile at 0x7fadc875af40>,
 <torch.profiler.profiler.profile at 0x7fadc7eec6a0>)

In [61]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3360 / 8192 (41.0%)
Greatest absolute difference: 0.005429118871688843 at index (8086,) (up to 1e-05 allowed)
Greatest relative difference: 0.019810057972665867 at index (7824,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:16 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:29 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:29 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(130.14064073562622, 8.063328010729265)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3134 / 8192 (38.3%)
Greatest absolute difference: 0.003577291965484619 at index (5807,) (up to 1e-05 allowed)
Greatest relative difference: 0.018509307933657716 at index (2885,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:30 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:40 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:40 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(94.88291025161743, 8.624523294264463)


(<torch.profiler.profiler.profile at 0x7fadc53b4a60>,
 <torch.profiler.profiler.profile at 0x7fadc4d6f790>)

In [62]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 321 / 1024 (31.3%)
Greatest absolute difference: 0.002856045961380005 at index (103,) (up to 1e-05 allowed)
Greatest relative difference: 0.015892281345223264 at index (375,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:41 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:42 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:42 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(9.938373565673828, 0.4330689299877122)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 318 / 1024 (31.1%)
Greatest absolute difference: 0.0024604201316833496 at index (334,) (up to 1e-05 allowed)
Greatest relative difference: 0.015153895173137647 at index (113,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:22:43 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:22:43 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:22:43 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.710178375244141, 0.3246004742777586)


(<torch.profiler.profiler.profile at 0x7fadc2fbe8b0>,
 <torch.profiler.profiler.profile at 0x7fadc1b2c460>)

In [63]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 637 / 2048 (31.1%)
Greatest absolute difference: 0.003785431385040283 at index (2029,) (up to 1e-05 allowed)
Greatest relative difference: 0.01398512411578297 at index (160,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:27 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:23:30 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:23:30 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(25.235610008239746, 0.38651390852919576)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 647 / 2048 (31.6%)
Greatest absolute difference: 0.0037910640239715576 at index (1734,) (up to 1e-05 allowed)
Greatest relative difference: 0.016010841453040887 at index (841,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:30 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:23:31 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:23:31 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(6.071324348449707, 0.48422270649552956)


(<torch.profiler.profiler.profile at 0x7fadbf74b670>,
 <torch.profiler.profiler.profile at 0x7fadbeb92250>)

In [64]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1293 / 4096 (31.6%)
Greatest absolute difference: 0.004175335168838501 at index (1236,) (up to 1e-05 allowed)
Greatest relative difference: 0.021651669329572177 at index (1851,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:35 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:23:40 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:23:40 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(45.87159872055054, 5.080251344514863)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1316 / 4096 (32.1%)
Greatest absolute difference: 0.004473268985748291 at index (1602,) (up to 1e-05 allowed)
Greatest relative difference: 0.022487578312769995 at index (2281,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:41 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:23:42 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:23:42 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.803684711456299, 0.657613505899235)


(<torch.profiler.profiler.profile at 0x7fadbd872880>,
 <torch.profiler.profiler.profile at 0x7fadbba0cc70>)

In [65]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2492 / 8192 (30.4%)
Greatest absolute difference: 0.00433880090713501 at index (1509,) (up to 1e-05 allowed)
Greatest relative difference: 0.019605897270292983 at index (4761,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:46 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:23:57 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:23:57 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(107.22804546356201, 2.143726845793026)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2507 / 8192 (30.6%)
Greatest absolute difference: 0.00366094708442688 at index (6791,) (up to 1e-05 allowed)
Greatest relative difference: 0.01757112764387071 at index (7437,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:23:58 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:03 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:03 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(45.858919620513916, 0.7067854475906188)


(<torch.profiler.profiler.profile at 0x7fadb97fcb80>,
 <torch.profiler.profiler.profile at 0x7fadb88c1b50>)

In [66]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 306 / 1024 (29.9%)
Greatest absolute difference: 0.0036117136478424072 at index (1014,) (up to 1e-05 allowed)
Greatest relative difference: 0.014455347348667122 at index (470,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:07 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:10 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:10 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(27.519211769104004, 0.2963214715464346)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 321 / 1024 (31.3%)
Greatest absolute difference: 0.003570854663848877 at index (199,) (up to 1e-05 allowed)
Greatest relative difference: 0.013518250975603316 at index (987,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:10 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:11 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:11 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.174968242645264, 0.6043094389212911)


(<torch.profiler.profiler.profile at 0x7fadb755e3a0>,
 <torch.profiler.profiler.profile at 0x7fadb2d6a310>)

In [67]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 630 / 2048 (30.8%)
Greatest absolute difference: 0.003431171178817749 at index (266,) (up to 1e-05 allowed)
Greatest relative difference: 0.01596878337303869 at index (1667,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:16 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:20 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:20 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(44.45408344268799, 0.9851899287923516)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 613 / 2048 (29.9%)
Greatest absolute difference: 0.0037209391593933105 at index (1160,) (up to 1e-05 allowed)
Greatest relative difference: 0.01773471707797356 at index (615,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:21 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:22 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:22 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(7.215080261230469, 1.0297961752257834)


(<torch.profiler.profiler.profile at 0x7fadb19a5340>,
 <torch.profiler.profiler.profile at 0x7fadad156f10>)

In [68]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1295 / 4096 (31.6%)
Greatest absolute difference: 0.004882842302322388 at index (663,) (up to 1e-05 allowed)
Greatest relative difference: 0.020928924305720877 at index (559,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:40 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:48 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:48 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(83.22283744812012, 8.23856554266058)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1244 / 4096 (30.4%)
Greatest absolute difference: 0.003281533718109131 at index (347,) (up to 1e-05 allowed)
Greatest relative difference: 0.014728080208955777 at index (317,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:24:49 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:24:50 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:24:50 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(12.52152681350708, 0.8992988081161002)


(<torch.profiler.profiler.profile at 0x7fadabe6ba90>,
 <torch.profiler.profiler.profile at 0x7fada7bd9ca0>)

In [69]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :8192
now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2585 / 8192 (31.6%)
Greatest absolute difference: 0.0039011240005493164 at index (4352,) (up to 1e-05 allowed)
Greatest relative difference: 0.02319946085394857 at index (19,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:27:39 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:27:56 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:27:56 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(172.68266916275024, 10.618603945312088)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2568 / 8192 (31.3%)
Greatest absolute difference: 0.004606366157531738 at index (1036,) (up to 1e-05 allowed)
Greatest relative difference: 0.01829328905707159 at index (3852,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-03 21:27:57 2044203:2044203 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-03 21:28:03 2044203:2044203 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-03 21:28:03 2044203:2044203 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(57.65246868133545, 7.8023043863368)


(<torch.profiler.profiler.profile at 0x7fad9da89850>,
 <torch.profiler.profiler.profile at 0x7fad9c240ee0>)